# Bootstrapping Confidence Intervals

## What we will accomplish in this notebook

We will two methods for obtaining a confidence interval from a bootstrapping distribution and mention one more.
- The percentile method
- The "reverse" method (also sometimes called "basic")
- We will just mention the $\textrm{BC}_{a}$ method ("**B**ias **C**orrected and **a**ccelerated") so that it is "on your radar".

## Introducing some notation

We have a sample stored in a dataframe $\textrm{df}$.  

* If we use a subscript of $0$ for a symbol (e.g. $S_0$) are indicating a "true" population level value.  
* If we use a hat over a symbol (e.g. $\hat{S}$) we are indicating an estimate using the original sample $\textrm{df}$. 
* If we use a tilde over a symbol (e.g. $\tilde{S} $) we are indicating an estimate from a bootstrap simulation or resample. 

For instance, we have a true parameter value of $\theta_0$ and our parameter estimate from this data will be $\hat{\theta}$.  When we simulated from the fitted model or resample from $\textrm{df}$ we generate bootstrap datasets $\widehat{\textrm{df}}_1, \widehat{\textrm{df}}_2, \widehat{\textrm{df}}_3, \dots ,\widehat{\textrm{df}}_B$. Similarly the corresponding re-estimates of the parameter be $\tilde{\theta}_1, \tilde{\theta}_2, \tilde{\theta}_3, \dots, \tilde{\theta}_B$.  Let $T$ be a statistic (i.e. a function which takes a sample and returns a real number).  We have  $\hat{t} = T(\textrm{df})$ and $\tilde{t}_i = T(\widehat{\textrm{df}}_i)$.

### The Percentile Method

When we use bootstrapping to compute confidence intervals for a statistic we are assuming that the distribution of $\tilde{t}$ is similar to the distribution of $\hat{t}$.  We want to approximate a $1-\alpha$ confidence interval for $t_0$.  Let $q_{\alpha/2}$ and $q_{1-\alpha/2}$ be the $\alpha/2$ and $1 - \alpha/2$ quantiles of $\tilde{t}$. 

The **percentile method** uses $(q_{\alpha/2}, q_{1-\alpha/2})$ as our estimate of the confidence interval. 

This is the method we used in the last notebook.  

It is generally **ill-advised** to use this method in practice, because it is easy to improve on the method.

### The "Reverse" Method

Compare the following two approximations:  $\hat{t} - t_0 \approx \tilde{t} - \hat{t}$ vs. $\hat{t} \approx \tilde{t}$.  Generally the first approximation will be closer than the second, for a given amount of data.  We can leverage this to give us somewhat more accurate confidence intervals as follows:

$$
\begin{align*}
1 - \alpha 
&= P\left( q_{\alpha/2} \lt \tilde{T} \lt q_{1- \alpha/2} \right)\\
&= P\left( q_{\alpha/2} - \hat{T} \lt \tilde{T} - \hat{T} \lt q_{1- \alpha/2}  - \hat{T}\right)\\
&\approx P\left( q_{\alpha/2} - \hat{T} \lt \hat{T} - t_0 \lt q_{1- \alpha/2}  - \hat{T}\right)\\
&= P\left( q_{\alpha/2} - 2\hat{T} \lt  - t_0 \lt q_{1- \alpha/2}  - 2 \hat{T}\right)\\
&= P\left( 2\hat{T} - q_{\alpha/2} \gt  t_0 \gt  2 \hat{T} - q_{1- \alpha/2} \right)\\
&= P\left( 2 \hat{T} - q_{1- \alpha/2} \lt t_0 \lt 2\hat{T} - q_{\alpha/2}   \right)\\
\end{align*}
$$

So we have obtained the confidence interval $(2 \hat{T} - q_{1- \alpha/2},  2\hat{T} - q_{\alpha/2})$.  I prefer to algebraically rewrite this in the form  $(\hat{T} - (q_{1- \alpha/2} - \hat{T}),  \hat{T} + (\hat{T} - q_{\alpha/2}))$ as it helps give me a visual understanding (subtracting is visually easier for me than doubling).  If the bootstrap sampling distribution is symmetric about $\hat{t}$ then this is equivalent to the percentile method, but in general it need not be.

Note that we are using the $1 - \alpha/2$ quantile on the **left** and the $\alpha/2$ quantile on the **right**.  That is why this is called the **reverse** method.  It will generally have coverage closer to the nominal level of $1-\alpha$ compared to the percentile method.  Since computing this requires essentially no additional work compared to the percentile method it is a good baseline method to use.  It is also often called the **basic** method.

Let's repeat a confidence interval calculation from the `1_Bootstrapping_Methods` notebook using this method to see how it would work.  We will regress price on living area and give a confidence interval for the slope.

In [6]:
import pandas as pd
import numpy as np 
from sklearn.linear_model import LinearRegression

df = pd.read_csv("../data/kc_house_data.csv")
nobs = df.shape[0]

# Fitting a linear regression model to the data
model = LinearRegression()
model.fit(df.sqft_living.values.reshape(-1,1), df.price)

# Resampling based bootstrap of the slope
slopes = []
resampled_model = LinearRegression()

for i in range(10000):
    resampled_df = df.sample(n=nobs, replace=True)
    resampled_model.fit(resampled_df.sqft_living.values.reshape(-1,1), resampled_df.price)
    slopes.append(resampled_model.coef_[0])

# Obtaining the quantiles
quantiles = np.quantile(slopes, [0.025, 0.975])

# Obtaining the lower and upper limits of the "reverse" confidence interval
lower = 2*model.coef_[0] - quantiles[1]
upper = 2*model.coef_[0] - quantiles[0]

# Comparing the percentile and reverse confidence intervals
print(f"Out point estimate of the slope is {model.coef_[0]}")
print(f"The percentile confidence interval is ({quantiles[0]},{quantiles[1]})")
print(f"The reverse confidence interval is ({lower},{upper})")

Out point estimate of the slope is 280.6235678974508
The percentile confidence interval is (269.84411454387265,291.6921899502655)
The reverse confidence interval is (269.5549458446361,291.403021251029)


We can see that these are very similar in this case, but they need not always be so close.

### The $\textrm{BC}_a$ method

We will just mention the bias corrected and accelerated bootstrap ($\textrm{BC}_a$). It accounts for both **bias** and **skewness** of the bootstrap distribution.  The resulting confidence interval is much more complicated to implement, but also has provably better coverage properties:  it is "second order accurate" rather than "first order accurate".  If you feel the need to use this in your work, one good option is [`scypi.stats.bootstrap`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bootstrap.html).  The original paper is [Efron 1987. Better bootstrap confidence intervals. Journal of the American Statistical Association 82 (397):171–85](https://www.tandfonline.com/doi/epdf/10.1080/01621459.1987.10478410?needAccess=true).

